# Recommendation System

# 1. Loading Dataset

In [77]:
import numpy as np
import pandas as pd
import ast
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [78]:
credits_data = pd.read_csv("/content/tmdb_5000_credits.csv", on_bad_lines='skip')
movies_data = pd.read_csv("/content/tmdb_5000_movies.csv", on_bad_lines='skip')

In [26]:
credits_data = pd.read_csv("/content/tmdb_5000_credits.csv", encoding='utf-8', on_bad_lines='skip')
movies_data = pd.read_csv("/content/tmdb_5000_movies.csv", encoding='utf-8', on_bad_lines='skip')


In [27]:
credits_data = pd.read_csv("/content/tmdb_5000_credits.csv", quotechar='"', on_bad_lines='skip')
movies_data = pd.read_csv("/content/tmdb_5000_movies.csv", quotechar='"', on_bad_lines='skip')


In [28]:
credits_data = pd.read_csv("/content/tmdb_5000_credits.csv", delimiter=',', on_bad_lines='skip')
movies_data = pd.read_csv("/content/tmdb_5000_movies.csv", delimiter=',', on_bad_lines='skip')


In [29]:
credits_data = pd.read_csv("/content/tmdb_5000_credits.csv", delimiter=',', quotechar='"', escapechar='\\', on_bad_lines='skip')
movies_data = pd.read_csv("/content/tmdb_5000_movies.csv", delimiter=',', quotechar='"', escapechar='\\', on_bad_lines='skip')


In [30]:
import pandas as pd

chunk_size = 500
chunks = pd.read_csv("/content/tmdb_5000_credits.csv", chunksize=chunk_size, delimiter=',', quotechar='"', escapechar='\\', on_bad_lines='skip')

# Combine chunks into a single DataFrame
credits_data = pd.concat(chunk for chunk in chunks)

chunks = pd.read_csv("/content/tmdb_5000_movies.csv", chunksize=chunk_size, delimiter=',', quotechar='"', escapechar='\\', on_bad_lines='skip')

# Combine chunks into a single DataFrame
movies_data = pd.concat(chunk for chunk in chunks)


In [79]:
credits_data

movie_id                                     title  \
0        19995                                    Avatar   
1          285  Pirates of the Caribbean: At World's End   
2       206647                                   Spectre   
3        49026                     The Dark Knight Rises   
4        49529                               John Carter   
...        ...                                       ...   
4798      9367                               El Mariachi   
4799     72766                                 Newlyweds   
4800    231617                 Signed, Sealed, Delivered   
4801    126186                          Shanghai Calling   
4802     25975                         My Date with Drew   

                                                   cast  \
0     [{"cast_id": 242, "character": "Jake Sully", "...   
1     [{"cast_id": 4, "character": "Captain Jack Spa...   
2     [{"cast_id": 1, "character": "James Bond", "cr...   
3     [{"cast_id": 2, "character": "Bruce Wayne / Ba...   
4     [{"cast_id": 5, "character": "John Carter", "c...   
...                                                 ...   
4798  [{"cast_id": 1, "character": "El Mariachi", "c...   
4799  [{"cast_id": 1, "character": "Buzzy", "credit_...   
4800  [{"cast_id": 8, "character": "Oliver O\u2019To...   
4801  [{"cast_id": 3, "character": "Sam", "credit_id...   
4802  [{"cast_id": 3, "character": "Herself", "credi...   

                                                   crew  
0     [{"credit_id": "52fe48009251416c750aca23", "de...  
1     [{"credit_id": "52fe4232c3a36847f800b579", "de...  
2     [{"credit_id": "54805967c3a36829b5002c41", "de...  
3     [{"credit_id": "52fe4781c3a36847f81398c3", "de...  
4     [{"credit_id": "52fe479ac3a36847f813eaa3", "de...  
...                                                 ...  
4798  [{"credit_id": "52fe44eec3a36847f80b280b", "de...  
4799  [{"credit_id": "52fe487dc3a368484e0fb013", "de...  
4800  [{"credit_id": "52fe4df3c3a36847f8275ecf", "de...  
4801  [{"credit_id": "52fe4ad9c3a368484e16a36b", "de...  
4802  [{"credit_id": "58ce021b9251415a390165d9", "de...  

[4803 rows x 4 columns]

In [80]:
movies_data

budget                                             genres  \
0     237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1     300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2     245000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
3     250000000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4     260000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
...         ...                                                ...   
4798     220000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4799       9000  [{"id": 35, "name": "Comedy"}, {"id": 10749, "...   
4800          0  [{"id": 35, "name": "Comedy"}, {"id": 18, "nam...   
4801          0                                                 []   
4802          0                [{"id": 99, "name": "Documentary"}]   

                                               homepage      id  \
0                           http://www.avatarmovie.com/   19995   
1          http://disney.go.com/disneypictures/pirates/     285   
2           http://www.sonypictures.com/movies/spectre/  206647   
3                    http://www.thedarkknightrises.com/   49026   
4                  http://movies.disney.com/john-carter   49529   
...                                                 ...     ...   
4798                                                NaN    9367   
4799                                                NaN   72766   
4800  http://www.hallmarkchannel.com/signedsealeddel...  231617   
4801                        http://shanghaicalling.com/  126186   
4802                                                NaN   25975   

                                               keywords original_language  \
0     [{"id": 1463, "name": "culture clash"}, {"id":...                en   
1     [{"id": 270, "name": "ocean"}, {"id": 726, "na...                en   
2     [{"id": 470, "name": "spy"}, {"id": 818, "name...                en   
3     [{"id": 849, "name": "dc comics"}, {"id": 853,...                en   
4     [{"id": 818, "name": "based on novel"}, {"id":...                en   
...                                                 ...               ...   
4798  [{"id": 5616, "name": "united states\u2013mexi...                es   
4799                                                 []                en   
4800  [{"id": 248, "name": "date"}, {"id": 699, "nam...                en   
4801                                                 []                en   
4802  [{"id": 1523, "name": "obsession"}, {"id": 224...                en   

                                original_title  \
0                                       Avatar   
1     Pirates of the Caribbean: At World's End   
2                                      Spectre   
3                        The Dark Knight Rises   
4                                  John Carter   
...                                        ...   
4798                               El Mariachi   
4799                                 Newlyweds   
4800                 Signed, Sealed, Delivered   
4801                          Shanghai Calling   
4802                         My Date with Drew   

                                               overview  popularity  \
0     In the 22nd century, a paraplegic Marine is di...  150.437577   
1     Captain Barbossa, long believed to be dead, ha...  139.082615   
2     A cryptic message from Bond’s past sends him o...  107.376788   
3     Following the death of District Attorney Harve...  112.312950   
4     John Carter is a war-weary, former military ca...   43.926995   
...                                                 ...         ...   
4798  El Mariachi just wants to play his guitar and ...   14.269792   
4799  A newlywed couple's honeymoon is upended by th...    0.642552   
4800  "Signed, Sealed, Delivered" introduces a dedic...    1.444476   
4801  When ambitious New York attorney Sam is sent t...    0.857008   
4802  Ever since the second grade when he first saw ... 

# 2. Merging Datasets into single

In [81]:
movies = movies_data.merge(credits_data, on='title')

In [82]:
movies

budget                                             genres  \
0     237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1     300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2     245000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
3     250000000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4     260000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
...         ...                                                ...   
4804     220000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4805       9000  [{"id": 35, "name": "Comedy"}, {"id": 10749, "...   
4806          0  [{"id": 35, "name": "Comedy"}, {"id": 18, "nam...   
4807          0                                                 []   
4808          0                [{"id": 99, "name": "Documentary"}]   

                                               homepage      id  \
0                           http://www.avatarmovie.com/   19995   
1          http://disney.go.com/disneypictures/pirates/     285   
2           http://www.sonypictures.com/movies/spectre/  206647   
3                    http://www.thedarkknightrises.com/   49026   
4                  http://movies.disney.com/john-carter   49529   
...                                                 ...     ...   
4804                                                NaN    9367   
4805                                                NaN   72766   
4806  http://www.hallmarkchannel.com/signedsealeddel...  231617   
4807                        http://shanghaicalling.com/  126186   
4808                                                NaN   25975   

                                               keywords original_language  \
0     [{"id": 1463, "name": "culture clash"}, {"id":...                en   
1     [{"id": 270, "name": "ocean"}, {"id": 726, "na...                en   
2     [{"id": 470, "name": "spy"}, {"id": 818, "name...                en   
3     [{"id": 849, "name": "dc comics"}, {"id": 853,...                en   
4     [{"id": 818, "name": "based on novel"}, {"id":...                en   
...                                                 ...               ...   
4804  [{"id": 5616, "name": "united states\u2013mexi...                es   
4805                                                 []                en   
4806  [{"id": 248, "name": "date"}, {"id": 699, "nam...                en   
4807                                                 []                en   
4808  [{"id": 1523, "name": "obsession"}, {"id": 224...                en   

                                original_title  \
0                                       Avatar   
1     Pirates of the Caribbean: At World's End   
2                                      Spectre   
3                        The Dark Knight Rises   
4                                  John Carter   
...                                        ...   
4804                               El Mariachi   
4805                                 Newlyweds   
4806                 Signed, Sealed, Delivered   
4807                          Shanghai Calling   
4808                         My Date with Drew   

                                               overview  popularity  \
0     In the 22nd century, a paraplegic Marine is di...  150.437577   
1     Captain Barbossa, long believed to be dead, ha...  139.082615   
2     A cryptic message from Bond’s past sends him o...  107.376788   
3     Following the death of District Attorney Harve...  112.312950   
4     John Carter is a war-weary, former military ca...   43.926995   
...                                                 ...         ...   
4804  El Mariachi just wants to play his guitar and ...   14.269792   
4805  A newlywed couple's honeymoon is upended by th...    0.642552   
4806  "Signed, Sealed, Delivered" introduces a dedic...    1.444476   
4807  When ambitious New York attorney Sam is sent t...    0.857008   
4808  Ever since the second grade when he first saw ... 

# 3. Feature Engineering

In [83]:
movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'movie_id', 'cast', 'crew'],
      dtype='object')

In [84]:
movies['original_language'].value_counts()

original_language
en    4510
fr      70
es      32
zh      27
de      27
hi      19
ja      16
it      14
ko      12
cn      12
ru      11
pt       9
da       7
sv       5
nl       4
fa       4
th       3
he       3
ta       2
cs       2
ro       2
id       2
ar       2
vi       1
sl       1
ps       1
no       1
ky       1
hu       1
pl       1
af       1
nb       1
tr       1
is       1
xx       1
te       1
el       1
Name: count, dtype: int64

In [85]:
movies['original_language'].unique()

array(['en', 'ja', 'fr', 'zh', 'es', 'ko', 'de', 'hi', 'ru', 'te', 'cn',
       'it', 'nl', 'ta', 'sv', 'th', 'da', 'xx', 'hu', 'cs', 'pt', 'is',
       'tr', 'nb', 'af', 'pl', 'he', 'ar', 'vi', 'ky', 'id', 'ro', 'fa',
       'no', 'sl', 'ps', 'el'], dtype=object)

In [86]:
movies = movies[['genres' , 'id' , 'keywords' , 'original_language' , 'title' , 'overview' , 'popularity' , 'cast' , 'crew']]

In [87]:
movies

genres      id  \
0     [{"id": 28, "name": "Action"}, {"id": 12, "nam...   19995   
1     [{"id": 12, "name": "Adventure"}, {"id": 14, "...     285   
2     [{"id": 28, "name": "Action"}, {"id": 12, "nam...  206647   
3     [{"id": 28, "name": "Action"}, {"id": 80, "nam...   49026   
4     [{"id": 28, "name": "Action"}, {"id": 12, "nam...   49529   
...                                                 ...     ...   
4804  [{"id": 28, "name": "Action"}, {"id": 80, "nam...    9367   
4805  [{"id": 35, "name": "Comedy"}, {"id": 10749, "...   72766   
4806  [{"id": 35, "name": "Comedy"}, {"id": 18, "nam...  231617   
4807                                                 []  126186   
4808                [{"id": 99, "name": "Documentary"}]   25975   

                                               keywords original_language  \
0     [{"id": 1463, "name": "culture clash"}, {"id":...                en   
1     [{"id": 270, "name": "ocean"}, {"id": 726, "na...                en   
2     [{"id": 470, "name": "spy"}, {"id": 818, "name...                en   
3     [{"id": 849, "name": "dc comics"}, {"id": 853,...                en   
4     [{"id": 818, "name": "based on novel"}, {"id":...                en   
...                                                 ...               ...   
4804  [{"id": 5616, "name": "united states\u2013mexi...                es   
4805                                                 []                en   
4806  [{"id": 248, "name": "date"}, {"id": 699, "nam...                en   
4807                                                 []                en   
4808  [{"id": 1523, "name": "obsession"}, {"id": 224...                en   

                                         title  \
0                                       Avatar   
1     Pirates of the Caribbean: At World's End   
2                                      Spectre   
3                        The Dark Knight Rises   
4                                  John Carter   
...                                        ...   
4804                               El Mariachi   
4805                                 Newlyweds   
4806                 Signed, Sealed, Delivered   
4807                          Shanghai Calling   
4808                         My Date with Drew   

                                               overview  popularity  \
0     In the 22nd century, a paraplegic Marine is di...  150.437577   
1     Captain Barbossa, long believed to be dead, ha...  139.082615   
2     A cryptic message from Bond’s past sends him o...  107.376788   
3     Following the death of District Attorney Harve...  112.312950   
4     John Carter is a war-weary, former military ca...   43.926995   
...                                                 ...         ...   
4804  El Mariachi just wants to play his guitar and ...   14.269792   
4805  A newlywed couple's honeymoon is upended by th...    0.642552   
4806  "Signed, Sealed, Delivered" introduces a dedic...    1.444476   
4807  When ambitious New York attorney Sam is sent t...    0.857008   
4808  Ever since the second grade when he first saw ...    1.929883   

                                                   cast  \
0     [{"cast_id": 242, "character": "Jake Sully", "...   
1     [{"cast_id": 4, "character": "Captain Jack Spa...   
2     [{"cast_id": 1, "character": "James Bond", "cr...   
3     [{"cast_id": 2, "character": "Bruce Wayne / Ba...   
4     [{"cast_id": 5, "character": "John Carter", "c...   
...                                                 ...   
4804  [{"cast_id": 1, "character": "El Mariachi", "c...   
4805  [{"cast_id": 1, "character": "Buzzy", "credit_...   
4806  [{"cast_id": 8, "character": "Oliver O\u2019To...   
4807  [{"cast_id": 3, "character": "Sam", "credit_id...   
4808  [{"cast_id": 3, "character": "Herself", "credi...   

                                                   crew  
0     [{"credit_id": "52fe48009251416c750aca23", "de...  
1     [{"credit_id": "52fe4232c3a36847f8

In [88]:
movies.duplicated().unique()

array([False])

In [89]:
movies.duplicated().sum()

0

In [90]:
movies.isnull().sum()

genres               0
id                   0
keywords             0
original_language    0
title                0
overview             3
popularity           0
cast                 0
crew                 0
dtype: int64

In [91]:
movies = movies.dropna()

In [92]:
movies.isnull().sum()

genres               0
id                   0
keywords             0
original_language    0
title                0
overview             0
popularity           0
cast                 0
crew                 0
dtype: int64

In [93]:
movies.iloc[0]['genres']

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [94]:
movies.iloc[0]['keywords']

'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

In [95]:
movies.iloc[0]['cast']

'[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 

In [96]:
movies.iloc[0]['crew']

'[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cameron"},

In [97]:
def get_values(value):
  values = ast.literal_eval(value)
  lst = []
  for dictionary in values:
    lst.append(dictionary['name'])
  return lst

In [98]:
movies["genres"] = movies["genres"].apply(lambda x : get_values(x))

In [99]:
movies["genres"]

0       [Action, Adventure, Fantasy, Science Fiction]
1                        [Adventure, Fantasy, Action]
2                          [Action, Adventure, Crime]
3                    [Action, Crime, Drama, Thriller]
4                [Action, Adventure, Science Fiction]
                            ...                      
4804                        [Action, Crime, Thriller]
4805                                [Comedy, Romance]
4806               [Comedy, Drama, Romance, TV Movie]
4807                                               []
4808                                    [Documentary]
Name: genres, Length: 4806, dtype: object

In [100]:
movies.iloc[0]['keywords']

'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

In [53]:
movies["keywords"] = movies["keywords"].apply(lambda x : get_values(x))

SyntaxError: unterminated triple-quoted string literal (detected at line 1) (<unknown>, line 1)

In [101]:
import pandas as pd
import ast

def get_values(value):
    try:
        # Safely evaluate the string to convert it into a Python object
        values = ast.literal_eval(value)
        # Extract the 'name' field from each dictionary in the list
        return [dictionary['name'] for dictionary in values]
    except (ValueError, SyntaxError) as e:
        # Log the error and the problematic value
        print(f"Error parsing value: {value}")
        print(f"Exception: {e}")
        return []  # Return an empty list or a default value

# Assuming 'movies' is your DataFrame and 'keywords' is a column in it
movies["keywords"] = movies["keywords"].apply(lambda x: get_values(x))

# Optionally, you can inspect the data to see if there are other problematic rows


In [102]:
movies["keywords"]

0       [culture clash, future, space war, space colon...
1       [ocean, drug abuse, exotic island, east india ...
2       [spy, based on novel, secret agent, sequel, mi...
3       [dc comics, crime fighter, terrorist, secret i...
4       [based on novel, mars, medallion, space travel...
                              ...                        
4804    [united states–mexico barrier, legs, arms, pap...
4805                                                   []
4806    [date, love at first sight, narration, investi...
4807                                                   []
4808            [obsession, camcorder, crush, dream girl]
Name: keywords, Length: 4806, dtype: object

In [103]:
movies.iloc[0]['keywords']

['culture clash',
 'future',
 'space war',
 'space colony',
 'society',
 'space travel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alien planet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'love affair',
 'anti war',
 'power relations',
 'mind and soul',
 '3d']

In [104]:
movies.iloc[0]['cast']

'[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 

In [105]:
# picking the first 5 cast from movies['cast'] to reduce parameters
def get_cast(value):
  values = ast.literal_eval(value)
  lst = []
  counter = 0
  for dictionary in values:
    if counter < 5:
      lst.append(dictionary['name'])
      counter += 1
    else:
      break
  return lst

In [106]:
movies["cast"] = movies["cast"].apply(lambda x : get_cast(x))

In [107]:
movies["cast"]

0       [Sam Worthington, Zoe Saldana, Sigourney Weave...
1       [Johnny Depp, Orlando Bloom, Keira Knightley, ...
2       [Daniel Craig, Christoph Waltz, Léa Seydoux, R...
3       [Christian Bale, Michael Caine, Gary Oldman, A...
4       [Taylor Kitsch, Lynn Collins, Samantha Morton,...
                              ...                        
4804    [Carlos Gallardo, Jaime de Hoyos, Peter Marqua...
4805    [Edward Burns, Kerry Bishé, Marsha Dietlein, C...
4806    [Eric Mabius, Kristin Booth, Crystal Lowe, Geo...
4807    [Daniel Henney, Eliza Coupe, Bill Paxton, Alan...
4808    [Drew Barrymore, Brian Herzlinger, Corey Feldm...
Name: cast, Length: 4806, dtype: object

In [108]:
movies.iloc[0]['cast']

['Sam Worthington',
 'Zoe Saldana',
 'Sigourney Weaver',
 'Stephen Lang',
 'Michelle Rodriguez']

In [109]:
def get_crew(value):
  values = ast.literal_eval(value)
  lst = []
  for dictionary in values:
    if dictionary['job'] == "Director":
      lst.append(dictionary['name'])
      break
  return lst

In [110]:
movies["crew"] = movies["crew"].apply(lambda x : get_crew(x))

In [111]:
movies.iloc[0]['crew']

['James Cameron']

In [112]:
movies.head()

genres      id  \
0  [Action, Adventure, Fantasy, Science Fiction]   19995   
1                   [Adventure, Fantasy, Action]     285   
2                     [Action, Adventure, Crime]  206647   
3               [Action, Crime, Drama, Thriller]   49026   
4           [Action, Adventure, Science Fiction]   49529   

                                            keywords original_language  \
0  [culture clash, future, space war, space colon...                en   
1  [ocean, drug abuse, exotic island, east india ...                en   
2  [spy, based on novel, secret agent, sequel, mi...                en   
3  [dc comics, crime fighter, terrorist, secret i...                en   
4  [based on novel, mars, medallion, space travel...                en   

                                      title  \
0                                    Avatar   
1  Pirates of the Caribbean: At World's End   
2                                   Spectre   
3                     The Dark Knight Rises   
4                               John Carter   

                                            overview  popularity  \
0  In the 22nd century, a paraplegic Marine is di...  150.437577   
1  Captain Barbossa, long believed to be dead, ha...  139.082615   
2  A cryptic message from Bond’s past sends him o...  107.376788   
3  Following the death of District Attorney Harve...  112.312950   
4  John Carter is a war-weary, former military ca...   43.926995   

                                                cast                 crew  
0  [Sam Worthington, Zoe Saldana, Sigourney Weave...      [James Cameron]  
1  [Johnny Depp, Orlando Bloom, Keira Knightley, ...     [Gore Verbinski]  
2  [Daniel Craig, Christoph Waltz, Léa Seydoux, R...         [Sam Mendes]  
3  [Christian Bale, Michael Caine, Gary Oldman, A...  [Christopher Nolan]  
4  [Taylor Kitsch, Lynn Collins, Samantha Morton,...     [Andrew Stanton]

In [113]:
movies

genres      id  \
0     [Action, Adventure, Fantasy, Science Fiction]   19995   
1                      [Adventure, Fantasy, Action]     285   
2                        [Action, Adventure, Crime]  206647   
3                  [Action, Crime, Drama, Thriller]   49026   
4              [Action, Adventure, Science Fiction]   49529   
...                                             ...     ...   
4804                      [Action, Crime, Thriller]    9367   
4805                              [Comedy, Romance]   72766   
4806             [Comedy, Drama, Romance, TV Movie]  231617   
4807                                             []  126186   
4808                                  [Documentary]   25975   

                                               keywords original_language  \
0     [culture clash, future, space war, space colon...                en   
1     [ocean, drug abuse, exotic island, east india ...                en   
2     [spy, based on novel, secret agent, sequel, mi...                en   
3     [dc comics, crime fighter, terrorist, secret i...                en   
4     [based on novel, mars, medallion, space travel...                en   
...                                                 ...               ...   
4804  [united states–mexico barrier, legs, arms, pap...                es   
4805                                                 []                en   
4806  [date, love at first sight, narration, investi...                en   
4807                                                 []                en   
4808          [obsession, camcorder, crush, dream girl]                en   

                                         title  \
0                                       Avatar   
1     Pirates of the Caribbean: At World's End   
2                                      Spectre   
3                        The Dark Knight Rises   
4                                  John Carter   
...                                        ...   
4804                               El Mariachi   
4805                                 Newlyweds   
4806                 Signed, Sealed, Delivered   
4807                          Shanghai Calling   
4808                         My Date with Drew   

                                               overview  popularity  \
0     In the 22nd century, a paraplegic Marine is di...  150.437577   
1     Captain Barbossa, long believed to be dead, ha...  139.082615   
2     A cryptic message from Bond’s past sends him o...  107.376788   
3     Following the death of District Attorney Harve...  112.312950   
4     John Carter is a war-weary, former military ca...   43.926995   
...                                                 ...         ...   
4804  El Mariachi just wants to play his guitar and ...   14.269792   
4805  A newlywed couple's honeymoon is upended by th...    0.642552   
4806  "Signed, Sealed, Delivered" introduces a dedic...    1.444476   
4807  When ambitious New York attorney Sam is sent t...    0.857008   
4808  Ever since the second grade when he first saw ...    1.929883   

                                                   cast                 crew  
0     [Sam Worthington, Zoe Saldana, Sigourney Weave...      [James Cameron]  
1     [Johnny Depp, Orlando Bloom, Keira Knightley, ...     [Gore Verbinski]  
2     [Daniel Craig, Christoph Waltz, Léa Seydoux, R...         [Sam Mendes]  
3     [Christian Bale, Michael Caine, Gary Oldman, A...  [Christopher Nolan]  
4     [Taylor Kitsch, Lynn Collins, Samantha Morton,...     [Andrew Stanton]  
...                                                 ...                  ...  
4804  [Carlos Gallardo, Jaime de Hoyos, Peter Marqua...   [Robert Rodriguez]  
4805  [Edward Burns, Kerry Bishé, Marsha Dietlein, C...       [Edward Burns]  
4806  [Eric Mabius, Kristin Booth, Crystal Lowe, Geo...        [Scott Smith]  
4807  [Daniel Henney, Eliza Coupe, Bill Paxton, Alan...        [Daniel Hsia]  
4808  [Drew Barrymore, Brian Herzlinger,

# 4. Tokenize

In [114]:
# split the sentence into word
movies["overview"] = movies["overview"].apply(lambda x : x.split(" "))

In [115]:
movies

genres      id  \
0     [Action, Adventure, Fantasy, Science Fiction]   19995   
1                      [Adventure, Fantasy, Action]     285   
2                        [Action, Adventure, Crime]  206647   
3                  [Action, Crime, Drama, Thriller]   49026   
4              [Action, Adventure, Science Fiction]   49529   
...                                             ...     ...   
4804                      [Action, Crime, Thriller]    9367   
4805                              [Comedy, Romance]   72766   
4806             [Comedy, Drama, Romance, TV Movie]  231617   
4807                                             []  126186   
4808                                  [Documentary]   25975   

                                               keywords original_language  \
0     [culture clash, future, space war, space colon...                en   
1     [ocean, drug abuse, exotic island, east india ...                en   
2     [spy, based on novel, secret agent, sequel, mi...                en   
3     [dc comics, crime fighter, terrorist, secret i...                en   
4     [based on novel, mars, medallion, space travel...                en   
...                                                 ...               ...   
4804  [united states–mexico barrier, legs, arms, pap...                es   
4805                                                 []                en   
4806  [date, love at first sight, narration, investi...                en   
4807                                                 []                en   
4808          [obsession, camcorder, crush, dream girl]                en   

                                         title  \
0                                       Avatar   
1     Pirates of the Caribbean: At World's End   
2                                      Spectre   
3                        The Dark Knight Rises   
4                                  John Carter   
...                                        ...   
4804                               El Mariachi   
4805                                 Newlyweds   
4806                 Signed, Sealed, Delivered   
4807                          Shanghai Calling   
4808                         My Date with Drew   

                                               overview  popularity  \
0     [In, the, 22nd, century,, a, paraplegic, Marin...  150.437577   
1     [Captain, Barbossa,, long, believed, to, be, d...  139.082615   
2     [A, cryptic, message, from, Bond’s, past, send...  107.376788   
3     [Following, the, death, of, District, Attorney...  112.312950   
4     [John, Carter, is, a, war-weary,, former, mili...   43.926995   
...                                                 ...         ...   
4804  [El, Mariachi, just, wants, to, play, his, gui...   14.269792   
4805  [A, newlywed, couple's, honeymoon, is, upended...    0.642552   
4806  ["Signed,, Sealed,, Delivered", introduces, a,...    1.444476   
4807  [When, ambitious, New, York, attorney, Sam, is...    0.857008   
4808  [Ever, since, the, second, grade, when, he, fi...    1.929883   

                                                   cast                 crew  
0     [Sam Worthington, Zoe Saldana, Sigourney Weave...      [James Cameron]  
1     [Johnny Depp, Orlando Bloom, Keira Knightley, ...     [Gore Verbinski]  
2     [Daniel Craig, Christoph Waltz, Léa Seydoux, R...         [Sam Mendes]  
3     [Christian Bale, Michael Caine, Gary Oldman, A...  [Christopher Nolan]  
4     [Taylor Kitsch, Lynn Collins, Samantha Morton,...     [Andrew Stanton]  
...                                                 ...                  ...  
4804  [Carlos Gallardo, Jaime de Hoyos, Peter Marqua...   [Robert Rodriguez]  
4805  [Edward Burns, Kerry Bishé, Marsha Dietlein, C...       [Edward Burns]  
4806  [Eric Mabius, Kristin Booth, Crystal Lowe, Geo...        [Scott Smith]  
4807  [Daniel Henney, Eliza Coupe, Bill Paxton, Alan...        [Daniel Hsia]  
4808  [Drew Barrymore, Brian Herzlinger,

In [116]:
# combine all data
movies["tag"] = movies["genres"] + movies["keywords"] + movies["cast"] + movies["crew"] + movies["overview"]

In [117]:
movies

genres      id  \
0     [Action, Adventure, Fantasy, Science Fiction]   19995   
1                      [Adventure, Fantasy, Action]     285   
2                        [Action, Adventure, Crime]  206647   
3                  [Action, Crime, Drama, Thriller]   49026   
4              [Action, Adventure, Science Fiction]   49529   
...                                             ...     ...   
4804                      [Action, Crime, Thriller]    9367   
4805                              [Comedy, Romance]   72766   
4806             [Comedy, Drama, Romance, TV Movie]  231617   
4807                                             []  126186   
4808                                  [Documentary]   25975   

                                               keywords original_language  \
0     [culture clash, future, space war, space colon...                en   
1     [ocean, drug abuse, exotic island, east india ...                en   
2     [spy, based on novel, secret agent, sequel, mi...                en   
3     [dc comics, crime fighter, terrorist, secret i...                en   
4     [based on novel, mars, medallion, space travel...                en   
...                                                 ...               ...   
4804  [united states–mexico barrier, legs, arms, pap...                es   
4805                                                 []                en   
4806  [date, love at first sight, narration, investi...                en   
4807                                                 []                en   
4808          [obsession, camcorder, crush, dream girl]                en   

                                         title  \
0                                       Avatar   
1     Pirates of the Caribbean: At World's End   
2                                      Spectre   
3                        The Dark Knight Rises   
4                                  John Carter   
...                                        ...   
4804                               El Mariachi   
4805                                 Newlyweds   
4806                 Signed, Sealed, Delivered   
4807                          Shanghai Calling   
4808                         My Date with Drew   

                                               overview  popularity  \
0     [In, the, 22nd, century,, a, paraplegic, Marin...  150.437577   
1     [Captain, Barbossa,, long, believed, to, be, d...  139.082615   
2     [A, cryptic, message, from, Bond’s, past, send...  107.376788   
3     [Following, the, death, of, District, Attorney...  112.312950   
4     [John, Carter, is, a, war-weary,, former, mili...   43.926995   
...                                                 ...         ...   
4804  [El, Mariachi, just, wants, to, play, his, gui...   14.269792   
4805  [A, newlywed, couple's, honeymoon, is, upended...    0.642552   
4806  ["Signed,, Sealed,, Delivered", introduces, a,...    1.444476   
4807  [When, ambitious, New, York, attorney, Sam, is...    0.857008   
4808  [Ever, since, the, second, grade, when, he, fi...    1.929883   

                                                   cast                 crew  \
0     [Sam Worthington, Zoe Saldana, Sigourney Weave...      [James Cameron]   
1     [Johnny Depp, Orlando Bloom, Keira Knightley, ...     [Gore Verbinski]   
2     [Daniel Craig, Christoph Waltz, Léa Seydoux, R...         [Sam Mendes]   
3     [Christian Bale, Michael Caine, Gary Oldman, A...  [Christopher Nolan]   
4     [Taylor Kitsch, Lynn Collins, Samantha Morton,...     [Andrew Stanton]   
...                                                 ...                  ...   
4804  [Carlos Gallardo, Jaime de Hoyos, Peter Marqua...   [Robert Rodriguez]   
4805  [Edward Burns, Kerry Bishé, Marsha Dietlein, C...       [Edward Burns]   
4806  [Eric Mabius, Kristin Booth, Crystal Lowe, Geo...        [Scott Smith]   
4807  [Daniel Henney, Eliza Coupe, Bill Paxton, Alan...        [Daniel Hsia]   
4808  [Drew Barrymore, Brian 

In [118]:
movies_tag = movies[["title" , "id" , "original_language" , "popularity" , "tag"]]

In [119]:
movies_tag

title      id original_language  \
0                                       Avatar   19995                en   
1     Pirates of the Caribbean: At World's End     285                en   
2                                      Spectre  206647                en   
3                        The Dark Knight Rises   49026                en   
4                                  John Carter   49529                en   
...                                        ...     ...               ...   
4804                               El Mariachi    9367                es   
4805                                 Newlyweds   72766                en   
4806                 Signed, Sealed, Delivered  231617                en   
4807                          Shanghai Calling  126186                en   
4808                         My Date with Drew   25975                en   

      popularity                                                tag  
0     150.437577  [Action, Adventure, Fantasy, Science Fiction, ...  
1     139.082615  [Adventure, Fantasy, Action, ocean, drug abuse...  
2     107.376788  [Action, Adventure, Crime, spy, based on novel...  
3     112.312950  [Action, Crime, Drama, Thriller, dc comics, cr...  
4      43.926995  [Action, Adventure, Science Fiction, based on ...  
...          ...                                                ...  
4804   14.269792  [Action, Crime, Thriller, united states–mexico...  
4805    0.642552  [Comedy, Romance, Edward Burns, Kerry Bishé, M...  
4806    1.444476  [Comedy, Drama, Romance, TV Movie, date, love ...  
4807    0.857008  [Daniel Henney, Eliza Coupe, Bill Paxton, Alan...  
4808    1.929883  [Documentary, obsession, camcorder, crush, dre...  

[4806 rows x 5 columns]

In [120]:
movies_tag["tag"][0]

['Action',
 'Adventure',
 'Fantasy',
 'Science Fiction',
 'culture clash',
 'future',
 'space war',
 'space colony',
 'society',
 'space travel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alien planet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'love affair',
 'anti war',
 'power relations',
 'mind and soul',
 '3d',
 'Sam Worthington',
 'Zoe Saldana',
 'Sigourney Weaver',
 'Stephen Lang',
 'Michelle Rodriguez',
 'James Cameron',
 'In',
 'the',
 '22nd',
 'century,',
 'a',
 'paraplegic',
 'Marine',
 'is',
 'dispatched',
 'to',
 'the',
 'moon',
 'Pandora',
 'on',
 'a',
 'unique',
 'mission,',
 'but',
 'becomes',
 'torn',
 'between',
 'following',
 'orders',
 'and',
 'protecting',
 'an',
 'alien',
 'civilization.']

In [146]:
# stopwords removal , stemming , puctuation and lowering
def process_tags(value):
  ps = PorterStemmer()
  tags = [ps.stem(word) for word in value if word not in stopwords.words('english')]
  text = " ".join(tags)
  text = re.sub(r'[^\w\s]' , "", text.lower())
  return text

In [147]:
movies_tag["proc_tags"] = movies_tag["tag"].apply(lambda x : process_tags(x))

In [148]:
movies_tag["proc_tags"]

0       action adventur fantasi science fict culture c...
1       adventur fantasi action ocean drug abus exotic...
2       action adventur crime spi based on novel secre...
3       action crime drama thriller dc comic crime fig...
4       action adventur science fict based on novel ma...
                              ...                        
4804    action crime thriller united statesmexico barr...
4805    comedi romanc edward burn kerry bishé marsha d...
4806    comedi drama romanc tv movi date love at first...
4807    daniel henney eliza coup bill paxton alan ruck...
4808    documentari obsess camcord crush dream girl dr...
Name: proc_tags, Length: 4806, dtype: object

In [149]:
cv = CountVectorizer(max_features = 5000 , stop_words = 'english')

In [150]:
vectors = cv.fit_transform(movies_tag["proc_tags"]).toarray()

In [151]:
vectors.shape

(4806, 5000)

In [152]:
cos_sin = cosine_similarity(vectors)

array([[1.        , 0.06495698, 0.06240865, ..., 0.04470646, 0.03431486,
        0.        ],
       [0.06495698, 1.        , 0.04003204, ..., 0.03823596, 0.        ,
        0.        ],
       [0.06240865, 0.04003204, 1.        , ..., 0.01836796, 0.08459099,
        0.        ],
       ...,
       [0.04470646, 0.03823596, 0.01836796, ..., 1.        , 0.02019893,
        0.03236348],
       [0.03431486, 0.        , 0.08459099, ..., 0.02019893, 1.        ,
        0.01863067],
       [0.        , 0.        , 0.        , ..., 0.03236348, 0.01863067,
        1.        ]])

In [153]:
cos_sin

array([[1.        , 0.06495698, 0.06240865, ..., 0.04470646, 0.03431486,
        0.        ],
       [0.06495698, 1.        , 0.04003204, ..., 0.03823596, 0.        ,
        0.        ],
       [0.06240865, 0.04003204, 1.        , ..., 0.01836796, 0.08459099,
        0.        ],
       ...,
       [0.04470646, 0.03823596, 0.01836796, ..., 1.        , 0.02019893,
        0.03236348],
       [0.03431486, 0.        , 0.08459099, ..., 0.02019893, 1.        ,
        0.01863067],
       [0.        , 0.        , 0.        , ..., 0.03236348, 0.01863067,
        1.        ]])

In [154]:
cos_sin.shape

(4806, 4806)

In [157]:
def recommeder_system(movie):
  movie_index = movies_tag[movies_tag['title'] == movie].index[0]
  dist = cos_sin[movie_index]
  movies_list = sorted(list(enumerate(dist)) , reverse=True , key = lambda x : x[1])[1:6]
  for movie in movies_list :
    print(movies_tag.iloc[movie[0]].title)

In [161]:
recommeder_system("Avatar")

Aliens
Moonraker
Silent Running
Treasure Planet
Lifeforce


In [162]:
movies_tag['title'][10]

'Superman Returns'

In [163]:
recommeder_system("Superman Returns")

Superman II
Superman III
Man of Steel
Superman
The Wolverine


In [164]:
import pickle

pickle.dump(cos_sin, open("similarity.pickle" , "wb"))
pickle.dump(movies_tag.to_dict(), open("movies_tag.pickle" , "wb"))

In [ ]:
# Content Based Filtering here